## Dependencies

In [ ]:
# Import machine learning dependencies

import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Import other dependencies

from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import pickle

In [ ]:
# Python SQL toolkit and Object Relational Mapper

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Import Pickled File - From Anu's connection to SQL

In [ ]:
# Open pickle
df = pickle.load(open('pickled_raws.pkl', 'rb'))

In [ ]:
df.head()

In [ ]:
# convert respondent_id back to column
df = df.reset_index()
df.head()

## Clean Data

In [ ]:
# Drop unnecessary columns and some other ones so the code actually runs
df = df.drop(columns=["denial_reason_1", "lien_status", "hoepa_status", "state_code", "purchaser_type", "respondent_id", "action_taken"]) 

In [ ]:
# Remove rows containing N/A, incomplete, or irrelevant information
## Only Home Purchase rows from loan_purpose column
df = df.loc[df["loan_purpose"] == "Home Purchase"]
df = df.drop(columns="loan_purpose")

## Remove Not Applicable from owner_occupancy
df = df.loc[df["owner_occupancy"] != "Not Applicable"]

## Remove Information not Provided and Not Applicable from Applicant and coapplicant sex
df = df.loc[df["applicant_sex"] != "Information not Provided"]
df = df.loc[df["applicant_sex"] != "Not Applicable"]
df = df.loc[df["co_applicant_sex"] != "Information not Provided"]
df = df.loc[df["co_applicant_sex"] != "Not Applicable"]

## Remove Information not Provided and Not Applicable from Applicant and coapplicant ethnicity
df = df.loc[df["applicant_ethnicity"] != "Not Applicable"]
df = df.loc[df["applicant_ethnicity"] != "Info not Provided by Applicant"]
df = df.loc[df["co_applicant_ethnicity"] != "Not Applicable"]
df = df.loc[df["co_applicant_ethnicity"] != "Info not Provided by Applicant"]

## Remove Not Applicable from Preapproval
df = df.loc[df["preapproval"] != "Not Applicable"]

## Remove Outliers from loan_amount_000s
df = df.loc[df["loan_amount_000s"] <= 1650]

# Restrict target variable to only approvals and denials
df = df.loc[df["action_taken_summary"] != 2]

In [ ]:
# Create new pickle file with final cleaned data
pickle.dump(df, open("Resources/final_pickle.pkl", "wb"))

In [ ]:
# check df again
df.count()

In [ ]:
df.head()

### Run Neural Network

In [ ]:
# Generate our categorical variable lists
action_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[action_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[action_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(action_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(action_cat,1)
df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 59
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 27

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=keras.layers.LeakyReLU(alpha=0.05)))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")